In [ ]:
%pip install -r requirements.txt

In [ ]:
# scrape https://www.poetryfoundation.org/poems/browse#page=1&sort_by=recently_added for 40,000 poems?

In [163]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging

In [235]:
with open('hi.txt', encoding='utf-8') as f:
    data = f.readlines()
bad_chars = ',& '
ndf = pd.DataFrame(columns=['Title', 'Author', 'Attributes', 'Poem'])
start = 0
for i in range(len(data)):
    data[i] = data[i][:-1]
    attr = None
    if data[i] == '<|endoftext|>':
        title_idx = data[start].find(' by ')
        title = data[start][1:title_idx-2]
        author = data[start][title_idx+4:]
        if author[-1] == ']':
            attr = author[author.find('[')+1:-1]
            author = author[:author.find('[')]
            attr = attr.replace('&', ' ').replace('   ', ',').lower()
            attr = attr.split(',')
            for j in range(len(attr)):
                attr[j] = attr[j].strip()
            while '' in attr:
                attr.remove('')
        pm = data[start+1:i]
        while '' in pm:
            pm.remove('')
        t = ''
        for j in range(len(pm)):
            t += pm[j].lower()+' NEWLINE\n '
        temp = [title, author, attr, t]
        ndf.loc[len(ndf)] = temp
        start = i+1


In [241]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oliver\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [242]:
Poems = ndf.Poem.map(lambda x: nltk.tokenize.word_tokenize(x))

In [ ]:
Poems[0]

In [216]:
ndf.to_csv('POEMS.csv', index=False)

In [116]:
for i in range(len(df)):
    if df.Author[i][-1] == ']':
        df.Title[i] += 

,Title,Author,Poem
0,Objects Used to Prop Open a Window,Michelle Menting,"['Dog bone, stapler,', 'cribbage board, garlic..."
1,The New Church,Lucia Cherciu,"['The old cupola glinted above the clouds, sho..."
2,Look for Me,Ted Kooser,"['Look for me under the hood', 'of that old Ch..."
3,Wild Life,Grace Cavalieri,"['Behind the silo, the Mother Rabbit', 'hunche..."
4,Umbrella,Connie Wanek,"['When I push your button', 'you fly off the h..."
...,...,...,...
13544,!,"Wendy Videlock [Relationships, Gay, Lesbian, Qu","['Dear Writers, I’m compiling the first in wha..."
13545,1 January 1965,"Joseph Brodsky [Living, Death, Growi","['The Wise Men will unlearn your name.', 'Abov..."
13546,1-800-FEAR,"Jody Gladding [Living, Social Commentari","[""We'd like to talk with you about fear they s..."
13547,0,"Hailey Leithauser [Arts & Sciences, Philosophy]","['Philosophic', 'in its complex, ovoid emptine..."


In [ ]:
# cleaning up poems
for i in range(len(df)):
    temp = df.Poem[i].strip()
    temp = temp.split('\r\r\n')
    temp = [t.strip() for t in temp]
    df.Poem[i] = [t for t in temp if t]
    df.Title[i] = df.Title[i].strip()

In [ ]:
# df.to_csv('clean_poems.csv', index=False, encoding='utf-8')

In [ ]:
# how should i go about generating new lines

In [ ]:
df = pd.read_csv('clean_poems.csv')

In [ ]:
df.Poem[13851]

In [ ]:
import fasttext

In [ ]:
model = fasttext.train_unsupervised(poems, model='skipgram')

In [ ]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)